In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [ ]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [ ]:
import sys
# 코랩의 경우 나눔 폰트를 설치합니다.
if 'google.colab' in sys.modules:
    !sudo apt-get -qq -y install fonts-nanum
    import matplotlib.font_manager as fm
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
    for fpath in font_files:
        fm.fontManager.addfont(fpath)

# 나눔 폰트를 사용합니다.
import matplotlib

matplotlib.rc('font', family='NanumBarunGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
!pip install keras-tuner

In [ ]:
from google.colab import files
uploaded = files.upload()  # 데이터 파일 업로드


In [ ]:
import pandas as pd

df = pd.read_csv("BostonHousingprice.csv")
X = df.iloc[:, :-1].values  # 입력 특성
y = df.iloc[:, -1].values  # 출력 값


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 데이터 로드
df = pd.read_csv("BostonHousingprice.csv")
X = df.iloc[:, :-1].values  # 입력 특성
y = df.iloc[:, -1].values  # 출력 값

# 데이터 분리
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = np.nan_to_num(scaler.fit_transform(X_train))
X_valid_scaled = np.nan_to_num(scaler.transform(X_valid))
X_test_scaled = np.nan_to_num(scaler.transform(X_test))
y_train = np.nan_to_num(y_train)
y_valid = np.nan_to_num(y_valid)
y_test = np.nan_to_num(y_test)

# 사용자 정의 HuberLoss 함수
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

# 모델 구조 정의
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256, step=16)
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-3, sampling="log")
    optimizer = hp.Choice("optimizer", values=["adam", "sgd"])

    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(X_train_scaled.shape[1],)))
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(1))

    loss = HuberLoss(threshold=hp.Float("threshold", min_value=0.5, max_value=5.0, step=0.5))
    model.compile(loss=loss, optimizer=optimizer, metrics=["mae"])
    return model

# 이 문단부터 하이퍼 파라미터
# Keras Tuner 설정
tuner = kt.Hyperband(
    build_model,
    objective="val_mae",
    max_epochs=20,
    factor=3,
    directory="tuner_dir",
    project_name="huber_loss_tuning",
    max_consecutive_failed_trials=10
)

# 하이퍼파라미터 검색 실행
tuner.search(
    X_train_scaled, y_train,
    validation_data=(X_valid_scaled, y_valid),
    verbose=2
)

# 최적의 하이퍼파라미터로 모델 재설정 및 훈련
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = build_model(best_hps)
history = model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=20)

# 모델 평가
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)

# 모델 저장
model.save("final_model.keras")

# 최종 평가 결과 출력
print(f"Test loss: {test_loss}")
print(f"Test MAE: {test_mae}")


In [ ]:
from google.colab import files

# 저장된 모델 다운로드
files.download("final_model.keras")


In [ ]:
results = {"Metric": ["Test Loss", "Test MAE"], "Value": [test_loss, test_mae]}
df = pd.DataFrame(results)
df.to_csv("results.csv", index=False)

# CSV 다운로드
files.download("results.csv")